In [ ]:
!pip install -U gensim

     |████████████████████████████████| 24.1 MB 1.8 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
import pandas as pd
import numpy as np
import gensim
from sklearn.linear_model import LogisticRegression
from gensim.models.doc2vec import TaggedDocument
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.svm import SVC
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
import multiprocessing
from sklearn import utils
from sklearn.model_selection import train_test_split
from gensim.parsing.preprocessing import remove_stopwords
from gensim.utils import simple_preprocess
from gensim.parsing.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import re
import nltk

import itertools
from gensim.similarities import MatrixSimilarity
from gensim.parsing.preprocessing import preprocess_documents
from gensim.parsing.preprocessing import preprocess_string

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True


# **For a resume recommend job descriptions.**



In [ ]:
resumeDf = pd.read_csv('resume_with_skills.csv')
resumeDf.drop(resumeDf.columns[[0]], axis = 1, inplace = True)
resumeDf #access row by id - 1

,id,category,resume,cleaned_resume,skills
0,1,software developer,SOFTWARE DEVELOPER INTERN ...,software developer intern summary skill profic...,"['buddy', 'linux', 'commerce', 'analytics', 'e..."
1,2,software developer,SOFTWARE DEVELOPER Professional...,software developer professional summary obtain...,"['nginx', 'linux', 'code review', 'issue track..."
2,3,software developer,GIS ANALYST/SOFTWARE DEVELOPER ...,gi analyst software developer summary contribu...,"['spatial analysis', 'framework', 'marketing',..."
3,4,software developer,SENIOR SOFTWARE DEVELOPER Caree...,senior software developer career focus work en...,"['commerce', 'java', 'junit', 'framework', 'co..."
4,5,software developer,SENIOR SOFTWARE DEVELOPER Summa...,senior software developer summary oracle certi...,"['linux', 'authentication', 'java', 'framework..."
...,...,...,...,...,...
436,437,java developer,GRADUATE RESEARCH ASSISTANT Pro...,graduate research assistant professional profi...,"['remote control', 'analytics', 'java', 'frame..."
437,438,java developer,GRADUATE RESEARCH ASSISTANT ...,graduate research assistant summary obtain int...,"['cryptographic protocol', 'linux', 'authentic..."
438,439,java developer,STUDENT ASSISTANT Qualificati...,student assistant qualification algorithm java...,"['linux', 'authentication', 'android studio', ..."
439,440,java developer,YW TEAM LEAD Summary ...,yw team lead summary motivated team lead posse...,"['linux', 'code review', 'java', 'react', 'sup..."


In [ ]:
jdDf = pd.read_csv('jd_with_skills.csv')
jdDf.drop(jdDf.columns[[0]], axis = 1, inplace = True)
jdDf

,id,category,jd,cleaned_jd,skills
0,1,cloud engineer,Company is seeking a Data Architect. The indiv...,company seeking data architect individual join...,"['data integrity', 'documentation', 'rotation'..."
1,2,cloud engineer,"Cloud Architect Lowell, MA 6+ months Resear...",cloud architect lowell 6 month research orient...,"['play', 'programming language', 'perl', 'reac..."
2,3,cloud engineer,"Cloud Architect - AWSJob Type: Contract, Contr...",cloud architect awsjob type contract contract ...,"['security', 'analytics', 'design', 'azure', '..."
3,4,cloud engineer,"Senior Software Architect (Perm)Charlestown, M...",senior software architect perm charlestown job...,"['angular', 'security', 'software engineering'..."
4,5,cloud engineer,"Our client, a well known and highly profitable...",client well known highly profitable software c...,"['security', 'design', 'web service', 'databas..."
...,...,...,...,...,...
501,502,java developer,Synechron is a recognized leader and expert in...,synechron recognized leader expert building bu...,"['design', 'information management', 'web serv..."
502,503,java developer,Note: For further information please contact S...,note information please contact sonal hatila s...,"['angular', 'oracle', 'framework', 'java', 'sp..."
503,504,java developer,"Synechron on behalf of our client, a global le...",synechron behalf client global leader distribu...,"['security', 'design', 'message queue', 'infor..."
504,505,java developer,Please contactPatrick TrainorCanyon Associates...,please contactpatrick trainorcanyon associates...,"['design', 'database', 'testing', 'java', 'sup..."


In [ ]:
jdsimDf = pd.read_csv('jd_similar.csv')
jdsimDf

,id,matched_ids,merged_skills
0,1,"[427.0, 407.0, 147.0, 143.0, 128.0, 158.0, 402...","{'component', 'monitoring', 'domain knowledge'..."
1,2,"[6.0, 488.0, 483.0, 41.0, 463.0, 475.0, 16.0, ...","{'component', 'business', 'ajax', 'apache tomc..."
2,3,"[36.0, 20.0, 46.0, 43.0, 31.0, 50.0, 5.0, 33.0...","{'openshift', 'monitoring', 'specification', '..."
3,4,"[98.0, 41.0, 51.0, 101.0, 250.0, 478.0, 88.0, ...","{'monitoring', 'data analysis', 'specification..."
4,5,"[31.0, 3.0, 50.0, 33.0, 46.0, 23.0, 40.0, 30.0...","{'component', 'monitoring', 'specification', '..."
...,...,...,...
501,502,"[504.0, 487.0, 230.0, 498.0, 3.0, 32.0, 477.0,...","{'component', 'business', 'machine learning', ..."
502,503,"[455.0, 463.0, 497.0, 487.0, 496.0, 459.0, 467...","{'business', 'junit', 'ajax', 'medium', 'mode'..."
503,504,"[502.0, 487.0, 93.0, 461.0, 396.0, 453.0, 442....","{'metadata', 'business', 'mode', 'hbase', 'mac..."
504,505,"[372.0, 488.0, 299.0, 463.0, 25.0, 483.0, 340....","{'component', 'monitoring', 'specification', '..."


In [ ]:
tagged_corpus = []
for i in range(len(jdDf)):
  tagged_corpus.append(TaggedDocument(jdDf['cleaned_jd'][i].split(' '), [jdDf['id'][i]]))

model = Doc2Vec(tagged_corpus,
                dm=0, 
                vector_size=500, 
                window=5, 
                min_count=1, 
                epochs=10, 
                hs=1)

In [ ]:
model.save('./d2vJD.model')

In [ ]:
from scipy import spatial
import ast
import math 

def get_cosine(a, b):
  result = 1 - spatial.distance.cosine(a, b)
  return result

def get_sim(given, required):
  score = 0
  given = ast.literal_eval(given)
  required = ast.literal_eval(required)
  given = set(given)
  for x in required:
    if x in given:
      score = score + 1
  result = 0
  if len(required) != 0:
    result = ((score*1.0)/len(required))
  return result

def get_count(cat, df, jds):

  c = 0
  d = 0
  i = 0

  for index, row in df.iterrows():
    id = row[0]
    i += 1
    if cat == jds['category'][id-1]:
      d += 1
      if(i==10):
        c = d

  return [c, d]

In [ ]:
list1 = [1, 3, 5, 7, 9]
list2 = [1, 3, 5, 7, 9]

result = pd.DataFrame()
result['id'] = ''
result['cos_10'] = ''
result['cos_20'] = ''
result['sim_10'] = ''
result['sim_20'] = ''

for i in list1:
  for j in list2:
    result['com_' + str(i) + '_' + str(j) + '_10' ] = ''
    result['com_' + str(i) + '_' + str(j) + '_20' ] = ''

result

,id,cos_10,cos_20,sim_10,sim_20,com_1_1_10,com_1_1_20,com_1_3_10,com_1_3_20,com_1_5_10,...,com_9_1_10,com_9_1_20,com_9_3_10,com_9_3_20,com_9_5_10,com_9_5_20,com_9_7_10,com_9_7_20,com_9_9_10,com_9_9_20


In [ ]:
tempDf = pd.DataFrame()
tempDf['id'] = ""
tempDf['cosine_sim'] = ""
tempDf['skills_sim'] = ""

for i in list1:
  for j in list2:
    tempDf['com_' + str(i) + '_' + str(j)] = ''

tempDf

,id,cosine_sim,skills_sim,com_1_1,com_1_3,com_1_5,com_1_7,com_1_9,com_3_1,com_3_3,...,com_7_1,com_7_3,com_7_5,com_7_7,com_7_9,com_9_1,com_9_3,com_9_5,com_9_7,com_9_9


In [ ]:
for k in range(len(resumeDf)):
  resume1 = resumeDf['cleaned_resume'][k].split(' ')
  resume1 = model.infer_vector(resume1)
  resume1_skills = resumeDf['skills'][k]
  cat = resumeDf['category'][k]
  idd = resumeDf['id'][k]

  tempDf = pd.DataFrame()
  tempDf['id'] = ""
  tempDf['cosine_sim'] = ""
  tempDf['skills_sim'] = ""

  for i in list1:
    for j in list2:
      tempDf['com_' + str(i) + '_' + str(j)] = ''

  for i in range(len(jdDf)):
    id = jdDf['id'][i]
    jd_vector = model.infer_vector(jdDf['cleaned_jd'][i].split(' '))
    cosine = get_cosine(resume1, jd_vector)
    skills = get_sim(resume1_skills, jdsimDf['merged_skills'][i])

    # print(cosine, skills)

    listt = [id, cosine, skills]

    for i in list1:
      for j in list2:
        listt.append((i*cosine + j*skills)/(i+j))
    
    tempDf.loc[len(tempDf.index)] = listt

  cc = get_count(cat, tempDf.sort_values(by = ['cosine_sim'], ascending = False).head(20), jdDf)
  cc_10 = cc[0]
  cc_20 = cc[1]

  dd = get_count(cat, tempDf.sort_values(by = ['skills_sim'], ascending = False).head(20), jdDf)
  dd_10 = dd[0]
  dd_20 = dd[1]

  listtt = [idd, cc_10, cc_20, dd_10, dd_20]

  for i in list1:
    for j in list2:
      pp = get_count(cat, tempDf.sort_values(by=['com_' + str(i) + '_' + str(j)], ascending = False).head(20), jdDf)
      listtt.append(pp[0])
      listtt.append(pp[1])

  result.loc[len(result.index)] = listtt
result

,id,cos_10,cos_20,sim_10,sim_20,com_1_1_10,com_1_1_20,com_1_3_10,com_1_3_20,com_1_5_10,...,com_9_1_10,com_9_1_20,com_9_3_10,com_9_3_20,com_9_5_10,com_9_5_20,com_9_7_10,com_9_7_20,com_9_9_10,com_9_9_20
0,1,0,4,0,1,1,4,0,1,0,...,0,4,0,4,0,4,0,4,1,4
1,2,0,7,0,3,0,4,0,3,0,...,3,7,3,6,0,6,0,5,0,4
2,3,0,2,2,3,0,3,2,3,3,...,0,2,0,2,0,3,0,3,0,3
3,4,0,7,0,3,0,3,2,3,2,...,4,7,4,8,0,6,0,3,0,3
4,5,0,3,0,2,0,2,0,2,0,...,0,3,0,2,0,1,0,1,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
436,437,0,10,0,5,0,11,0,8,0,...,0,12,6,11,0,11,6,9,0,11
437,438,0,5,0,3,0,7,0,4,0,...,0,5,0,6,0,7,4,6,0,7
438,439,0,9,2,4,0,9,3,5,0,...,0,9,6,10,7,10,6,11,0,9
439,440,4,6,3,5,4,5,3,5,3,...,4,7,0,8,0,8,4,6,4,5


In [ ]:
get_count(resumeDf['category'][0], tempDf.sort_values(by = ['skills_sim'], ascending = False).head(20), jdDf)


[2, 3]

In [ ]:
result.to_csv('resume_recommend.csv', index = False)

In [ ]:
for row in t2.head(20).iterrows():
  print(jdDf['category'][row[0]])

NameError: ignored

In [ ]:
resumeDf['category'][200]